# 使用 Featuretools 来自动化特征工程
## 比较了 gdb 默认参数模型与 TPOT 得到 gdb 模型

- 通过之前的尝试，可以看到通过 TPOT 自动化生成的模型表现并不好，甚至不如基准模型
- 与最开始，手工构造特征工程后，使用的 Decision Tree 和 Adaboost 模型也有很大差距

- 手工构造特征工程的时候，主要是构造了 store average sales 等新的特征

- 因此，有理由相信，继续挖掘特征可以提高模型表现
- 但是，手动构造特征是一个非常耗时间的过程，同时，由于个人知识面、分析能力的差异，完全有可能漏掉很多需要复杂过程构造的特征
- 这里使用的是Featuretools,是一个开源 Python 库，项目地址https://docs.featuretools.com/#
- 学习参考的内容来源于https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np


# Problem Description

- The task is a supervised regression problem: given sales data of Rossmann - a German chain drug store , I want to build a model that can predict the furture sales of Rossmann for the next 6 weeks. 

- Before I switch to the automated machine learning method, I have tried some complete machine learning model such as decision tree and Adaboost totally by hand. That is , the data cleaning, feature engineering, model selection and parameters tuning (although have not done yet for decision tree and adboost ) are done manualy

## Dataset

- Colab is able to retrieve data from GitHub directely

- store, train and test data sets are given but features and lables need to be splitted

- **Pay attention that test.csv does not include** ***Customers***

In [2]:
# read data from GitHub

train = pd.read_csv("inputs/train.csv")
test = pd.read_csv("inputs/test.csv")
store = pd.read_csv("inputs/store.csv")

print('Training data shape: ', train.shape)
print('testing data shape: ', test.shape)
print('Store data shape: ', store.shape)

/Users/limao/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Training data shape:  (1017209, 9)
testing data shape:  (41088, 8)
Store data shape:  (1115, 10)


## data cleaning

In [3]:
#train中去除open=1但是sales=0的数据

train = train.loc[train['Sales']>0]

print('{} train datas were deleted'.format(1017209 - len(train)))

172871 train datas were deleted


In [4]:
#train 中StateHoliday 将字符与数字0混淆，存在5个不同值

train.StateHoliday = train.StateHoliday.map({'0':'0',0:'0','a':'a','b':'b','c':'c'})

print('StateHoliday unique values {}'.format(len(train.StateHoliday.unique())))


StateHoliday unique values 4


In [5]:
#store中的NaN用0填充,表示没有竞争或者没有没有促销

store.fillna(0,inplace = True)

#test中的NaN用1填充,因为其中的NaN 全部是 Open，如果Open=0 则没有预测的必要了

test.fillna(1,inplace = True)

In [6]:
store.head()
store.isnull().any()

Store                        False
StoreType                    False
Assortment                   False
CompetitionDistance          False
CompetitionOpenSinceMonth    False
CompetitionOpenSinceYear     False
Promo2                       False
Promo2SinceWeek              False
Promo2SinceYear              False
PromoInterval                False
dtype: bool

In [7]:
train.head()
train.isnull().any()

Store            False
DayOfWeek        False
Date             False
Sales            False
Customers        False
Open             False
Promo            False
StateHoliday     False
SchoolHoliday    False
dtype: bool

In [8]:
test.head()
test.isnull().any()

Id               False
Store            False
DayOfWeek        False
Date             False
Open             False
Promo            False
StateHoliday     False
SchoolHoliday    False
dtype: bool

In [9]:
train['StateHoliday'].unique()

array(['0', 'a', 'b', 'c'], dtype=object)

## 使用 Featuretools 进行特征工程
特征工程中，构造新特征最主要的两项操作就是 transformation 和 aggregation. 前者是在单一 dataframe 上进行的操作，后者是在多个 dataframe 上进行的操作，非常类似于数据链表。其实，在 adaboost 模型中，构造单个store的平均值数据，已经用到了aggregation。但是，是通过手动的方式。featuretools 就是把一些列这些操作自动化，不仅可以节省很多时间，同时还可以挖掘出大量手动很难或者没有时间去构造的特征。正好这个项目中有 store、train 两个dataframe，因而很适合使用 featuretools.

## Entities and EntitySets

- 这两个是 featuretools 中的基本概念，简单讲其实一个 entity 就是一个dataframe。具体到本问题中，就是 test、store 

- entitysets 就是 entity 的集合。可以当做 Python 中另外一种数据类型

In [10]:
import featuretools as ft
# Create new entityset
es = ft.EntitySet(id = 'sales')

### 对于 store 有 unique index，那就是 ‘Store' ，因而可以用来构造 entity

In [11]:
# Create an entity from the store dataframe
# This dataframe already has an index 
es = es.entity_from_dataframe(entity_id = 'store', dataframe = store, 
                              variable_types = {'StoreType': ft.variable_types.Categorical,
                                                 'Assortment': ft.variable_types.Categorical,
                                                'Promo2': ft.variable_types.Categorical,
                                                'PromoInterval': ft.variable_types.Categorical},
                              index = 'Store')

In [12]:
es

Entityset: sales
  Entities:
    store [Rows: 1115, Columns: 10]
  Relationships:
    No relationships

### 对于 train dataframe，没有 unique index，所以必须添加 index

In [13]:
# Create an entity from the train dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'train', 
                              dataframe = train,
                              variable_types = {'Promo': ft.variable_types.Categorical,
                                                 'StateHoliday': ft.variable_types.Categorical,
                                                 'SchoolHoliday': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'train_id')

## Table relationship
简答说就是两张表中的“一对多”关系。具体在本项目中，store dataframe 是 train dataframe 的父表。因为，在store dataframe中一个store只有一行，而在train datafram 中一个记录了很多天的销售记录。父表与子表，通过共有的一个变量链接起来，在这里就 'Store'.

In [14]:
# Relationship between store and train
r_sales_previous = ft.Relationship(es['store']['Store'],
                                    es['train']['Store'])

In [15]:
# Add the relationship to the entity set
es = es.add_relationship(r_sales_previous)

In [16]:
es

Entityset: sales
  Entities:
    store [Rows: 1115, Columns: 10]
    train [Rows: 844338, Columns: 10]
  Relationships:
    train.Store -> store.Store

### 现在 entityset 就包含了 2个entity（dataframe）以及两张表之间的关系

## 现在就可以做 deep feature synthesis
- 其实就是合成新的特征，通过 aggregation 以及 transformation等方式。
- **最初的尝试中，没有设置agg/trans primitives**，然后导致最后的feature 有52个，太多了。

In [17]:
#ft.primitives.list_primitives()

In [18]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='train', 
                                 agg_primitives = ['mean'],
                                 #trans_primitives = ['DayOfWeek'],
                                 max_depth = 2)

features.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,DAY(Date),YEAR(Date),...,store.Promo2SinceWeek,store.Promo2SinceYear,store.StoreType,store.Assortment,store.Promo2,store.PromoInterval,store.MEAN(train.DayOfWeek),store.MEAN(train.Sales),store.MEAN(train.Customers),store.MEAN(train.Open)
train_id,,,,,,,,,,,,,,,,,,,,,
0,1,5,5263,555,1,1,0,1,31,2015,...,0.0,0.0,c,a,0,0,3.504481,4759.096031,564.049936,1
1,2,5,6064,625,1,1,0,1,31,2015,...,13.0,2010.0,a,a,1,"Jan,Apr,Jul,Oct",3.506378,4953.900510,583.998724,1
2,3,5,8314,821,1,1,0,1,31,2015,...,14.0,2011.0,a,a,1,"Jan,Apr,Jul,Oct",3.499358,6942.568678,750.077022,1
3,4,5,13995,1498,1,1,0,1,31,2015,...,0.0,0.0,c,c,0,0,3.506378,9638.401786,1321.752551,1
4,5,5,4822,559,1,1,0,1,31,2015,...,0.0,0.0,a,a,0,0,3.508344,4676.274711,537.340180,1


In [19]:
features['StateHoliday'].unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [20]:
avgperday = features.groupby(['Store', 'DayOfWeek'])['Sales'].mean().reset_index()
avgperday.columns = ['Store','DayOfWeek','AvgPerDay']
avgperday.head()

,Store,DayOfWeek,AvgPerDay
0,1,1,5177.968750
1,1,2,4685.626866
2,1,3,4555.712121
3,1,4,4457.838710
4,1,5,4726.480620


In [21]:
avgperday.describe()

,Store,DayOfWeek,AvgPerDay
count,6723.000000,6723.000000,6723.000000
mean,558.149189,3.517180,6945.707082
std,321.771361,1.721228,2588.524207
min,1.000000,1.000000,935.559701
25%,280.000000,2.000000,5254.550825
50%,558.000000,4.000000,6578.250000
75%,837.000000,5.000000,8125.751642
max,1115.000000,7.000000,28450.328358


In [22]:
features = pd.merge(features,avgperday,on =['Store','DayOfWeek'],how ='left')

In [23]:
list(features)

['Store',
 'DayOfWeek',
 'Sales',
 'Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'DAY(Date)',
 'YEAR(Date)',
 'MONTH(Date)',
 'WEEKDAY(Date)',
 'store.CompetitionDistance',
 'store.CompetitionOpenSinceMonth',
 'store.CompetitionOpenSinceYear',
 'store.Promo2SinceWeek',
 'store.Promo2SinceYear',
 'store.StoreType',
 'store.Assortment',
 'store.Promo2',
 'store.PromoInterval',
 'store.MEAN(train.DayOfWeek)',
 'store.MEAN(train.Sales)',
 'store.MEAN(train.Customers)',
 'store.MEAN(train.Open)',
 'AvgPerDay']

In [24]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 26 columns):
Store                              844338 non-null int64
DayOfWeek                          844338 non-null int64
Sales                              844338 non-null int64
Customers                          844338 non-null int64
Open                               844338 non-null int64
Promo                              844338 non-null int64
StateHoliday                       844338 non-null object
SchoolHoliday                      844338 non-null int64
DAY(Date)                          844338 non-null int64
YEAR(Date)                         844338 non-null int64
MONTH(Date)                        844338 non-null int64
WEEKDAY(Date)                      844338 non-null int64
store.CompetitionDistance          844338 non-null float64
store.CompetitionOpenSinceMonth    844338 non-null float64
store.CompetitionOpenSinceYear     844338 non-null float64
store.Promo2SinceWeek      

## 剔除掉没意义的 feature
- DAY(Date)\YEAR(Date)\MONTH(Date)\WEEKDAY(Date)\Cusoterms ,因为test数据中没有date
- store.MEAN(train.DayOfWeek)\store.MEAN(train.Open),也没有意义，因为我们需要预测的肯定都是开着的商店。考察一周平均开几天没有意义

In [25]:
features.drop({'Customers','DAY(Date)','YEAR(Date)','MONTH(Date)','WEEKDAY(Date)','store.MEAN(train.DayOfWeek)','store.MEAN(train.Open)'},
              inplace = True, axis =1)

In [26]:
features.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,store.CompetitionDistance,store.CompetitionOpenSinceMonth,store.CompetitionOpenSinceYear,store.Promo2SinceWeek,store.Promo2SinceYear,store.StoreType,store.Assortment,store.Promo2,store.PromoInterval,store.MEAN(train.Sales),store.MEAN(train.Customers),AvgPerDay
0,1,5,5263,1,1,0,1,1270.0,9.0,2008.0,0.0,0.0,c,a,0,0,4759.096031,564.049936,4726.480620
1,2,5,6064,1,1,0,1,570.0,11.0,2007.0,13.0,2010.0,a,a,1,"Jan,Apr,Jul,Oct",4953.900510,583.998724,4669.589147
2,3,5,8314,1,1,0,1,14130.0,12.0,2006.0,14.0,2011.0,a,a,1,"Jan,Apr,Jul,Oct",6942.568678,750.077022,7193.203125
3,4,5,13995,1,1,0,1,620.0,9.0,2009.0,0.0,0.0,c,c,0,0,9638.401786,1321.752551,9470.410853
4,5,5,4822,1,1,0,1,29910.0,4.0,2015.0,0.0,0.0,a,a,0,0,4676.274711,537.340180,4858.054688


In [27]:
# Sales 是预测值 label，需要从 features 中剔除

label = features['Sales'].to_frame()

features.drop('Sales',axis =1, inplace = True)

label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 1 columns):
Sales    844338 non-null int64
dtypes: int64(1)
memory usage: 12.9 MB


### one-hot encoding categorical features

- 非数值特征需要进行独热编码

In [28]:
features = pd.get_dummies(features,
                         prefix=['StateHoliday','store.StoreType', 'store.Assortment','store.PromoInterval'], 
                         drop_first=True)

#不知道为什么在这里get_dummy 把 StateHoliday = 0 忽略了，导致独热编码后只有3个种类，因而改成用map
#features.StateHoliday = train.StateHoliday.map({'0':0,'a':1,'b':2,'c':2})

In [29]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 25 columns):
Store                                   844338 non-null int64
DayOfWeek                               844338 non-null int64
Open                                    844338 non-null int64
Promo                                   844338 non-null int64
SchoolHoliday                           844338 non-null int64
store.CompetitionDistance               844338 non-null float64
store.CompetitionOpenSinceMonth         844338 non-null float64
store.CompetitionOpenSinceYear          844338 non-null float64
store.Promo2SinceWeek                   844338 non-null float64
store.Promo2SinceYear                   844338 non-null float64
store.Promo2                            844338 non-null int64
store.MEAN(train.Sales)                 844338 non-null float64
store.MEAN(train.Customers)             844338 non-null float64
AvgPerDay                               844338 non-null float64
State

In [30]:
list(features)

['Store',
 'DayOfWeek',
 'Open',
 'Promo',
 'SchoolHoliday',
 'store.CompetitionDistance',
 'store.CompetitionOpenSinceMonth',
 'store.CompetitionOpenSinceYear',
 'store.Promo2SinceWeek',
 'store.Promo2SinceYear',
 'store.Promo2',
 'store.MEAN(train.Sales)',
 'store.MEAN(train.Customers)',
 'AvgPerDay',
 'StateHoliday_a',
 'StateHoliday_b',
 'StateHoliday_c',
 'store.StoreType_b',
 'store.StoreType_c',
 'store.StoreType_d',
 'store.Assortment_b',
 'store.Assortment_c',
 'store.PromoInterval_Feb,May,Aug,Nov',
 'store.PromoInterval_Jan,Apr,Jul,Oct',
 'store.PromoInterval_Mar,Jun,Sept,Dec']

## 用 Train 数据训练测试模型

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = [
    'Store',
    'DayOfWeek',
    'Open',
    'Promo',
    'SchoolHoliday',
    'store.CompetitionDistance',
    'store.CompetitionOpenSinceMonth',
    'store.CompetitionOpenSinceYear',
    'store.Promo2SinceWeek',
    'store.Promo2SinceYear',
    'store.Promo2',
    'store.MEAN(train.Sales)',
    'store.MEAN(train.Customers)',
    'AvgPerDay',
    'StateHoliday_a',
    'StateHoliday_b',
    'StateHoliday_c',
    'store.StoreType_b',
    'store.StoreType_c',
    'store.StoreType_d',
    'store.Assortment_b',
    'store.Assortment_c',
    'store.PromoInterval_Feb,May,Aug,Nov',
    'store.PromoInterval_Jan,Apr,Jul,Oct',
    'store.PromoInterval_Mar,Jun,Sept,Dec'  
]




X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.33, random_state=42)



In [32]:
model_raw = GradientBoostingRegressor(verbose=True)

In [33]:
model_raw.fit(X_train, y_train)
model_raw.score(X_test, y_test)

/Users/limao/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1     8312080.8969            1.36m
         2     7222347.1261            1.30m
         3     6317637.9414            1.29m
         4     5567852.1264            1.25m
         5     4945819.3299            1.23m
         6     4428949.4788            1.21m
         7     4001276.9480            1.21m
         8     3641994.6608            1.18m
         9     3347721.4457            1.16m
        10     3098587.9878            1.14m
        20     2026164.4450            1.01m
        30     1747205.3477           52.72s
        40     1636857.7073           45.61s
        50     1598265.1676           37.64s
        60     1583693.9427           29.86s
        70     1575377.6054           21.98s
        80     1567516.8580           14.47s
        90     1564171.8089            7.14s
       100     1562723.5272            0.00s


0.8391827266559431

In [34]:
model_raw.get_params()

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': True,
 'warm_start': False}

In [35]:
model_tpot = GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss='quantile', max_depth=3, 
                                  max_features=0.05, min_samples_leaf=7, min_samples_split=5, 
                                  n_estimators=100, subsample=0.25,verbose=True)

In [36]:
model_tpot.fit(X_train, y_train)
model_tpot.score(X_test, y_test)

/Users/limao/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss      OOB Improve   Remaining Time 
         1         669.2807           1.9193           18.22s
         2         666.6339           5.6871           15.76s
         3         657.7496           3.5744           14.63s
         4         653.9280           5.5247           13.96s
         5         640.6693          18.3276           13.66s
         6         632.1298           2.5789           13.79s
         7         628.8948           3.8721           13.46s
         8         594.8622          36.4403           13.16s
         9         570.8917          25.1174           12.85s
        10         564.9847           2.6484           12.54s
        20         499.3863           2.9535           10.86s
        30         438.7711           0.7669            9.34s
        40         398.3518           2.6089            7.94s
        50         369.6777           0.3499            6.58s
        60         347.4093           2.1844            5.25s
       

0.39936925474324747

In [37]:
model_tpot.get_params()

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'quantile',
 'max_depth': 3,
 'max_features': 0.05,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 7,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': None,
 'subsample': 0.25,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': True,
 'warm_start': False}

## 从上面可以看到，同一个 feature 组合下， TPOT 得到的 gdb 表现是非常差的！
## 必须找到问题，调试 TPOT 的 gdb！



In [38]:
model_up1 = GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss='quantile', max_depth=10,  
                                  n_estimators=500, subsample=1,verbose=True)

In [39]:
model_up1.fit(X_train, y_train)
model_up1.score(X_test, y_test)

/Users/limao/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1         609.1388          160.02m
         2         554.4160          161.15m
         3         506.8411          162.12m
         4         465.6653          161.42m
         5         430.6213          159.27m
         6         400.7386          158.60m
         7         375.6860          157.64m
         8         354.5749          157.60m
         9         336.7202          157.49m
        10         321.4441          157.46m
        20         250.6653          154.57m
        30         234.2754          152.86m
        40         229.8251          146.89m
        50         228.2325          134.06m
        60         227.5817          120.67m
        70         227.3962          107.53m
        80         227.2396           95.91m
        90         227.0308           86.96m
       100         226.8428           79.68m
       200         226.0860           40.28m
       300         225.8448           22.02m
       40

0.6109330499609221

In [40]:
model_up2 = GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss='ls', max_depth=10,  
                                  n_estimators=200, subsample=1,verbose=True)

model_up2.fit(X_train, y_train)
model_up2.score(X_test, y_test)

/Users/limao/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1     8091184.0787           76.13m
         2     6837882.4066           74.77m
         3     5822110.9555           74.85m
         4     4998567.7363           74.54m
         5     4330886.1082           73.95m
         6     3788718.3236           73.78m
         7     3349147.6477           73.70m
         8     2992339.9750           73.34m
         9     2702794.2570           72.90m
        10     2467839.2753           72.54m
        20     1571767.5045           68.22m
        30     1448591.4946           63.14m
        40     1424456.8156           57.32m
        50     1415791.8773           51.33m
        60     1410948.4728           45.30m
        70     1408597.3538           39.20m
        80     1406722.0713           33.93m
        90     1404460.7744           29.86m
       100     1402220.6844           26.33m
       200     1389907.1035            0.00s


0.8462925990609441

## Preparing testing data
- 注意到 test 中 ’Id‘是多余的
- test 中 StateHoliday 的种类少了3类，在进行独热编码后，需要补全

In [134]:
len(test['Id'])

41088

In [135]:
len(store['Store'])

1115

## store 中增加了平均数据

In [136]:
avg_store = features[['Store','store.MEAN(train.Sales)','store.MEAN(train.Customers)']]

avg_store.head()

,Store,store.MEAN(train.Sales),store.MEAN(train.Customers)
0,1,4759.096031,564.049936
1,2,4953.900510,583.998724
2,3,6942.568678,750.077022
3,4,9638.401786,1321.752551
4,5,4676.274711,537.340180


In [137]:
len(avg_store['Store'].unique())

1115

In [138]:
avg_store.drop_duplicates(inplace = True)

In [139]:
len(avg_store['Store'])

1115

In [140]:
len(store['Store'])

1115

In [141]:
#为什么上面 store 变位8万多行了？

store = pd.merge(store,avg_store,on ='Store',how ='left')
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,store.MEAN(train.Sales),store.MEAN(train.Customers)
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,4759.096031,564.049936
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4953.900510,583.998724
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",6942.568678,750.077022
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0,9638.401786,1321.752551
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0,4676.274711,537.340180


In [142]:
store.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,store.MEAN(train.Sales),store.MEAN(train.Customers)
count,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000
mean,558.00000,5390.358744,4.930942,1370.939013,0.512108,12.083408,1030.239462,6934.631987,754.565433
std,322.01708,7657.973705,4.284924,935.467654,0.500078,15.542241,1006.038782,2383.982506,353.389229
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2703.736573,240.183099
25%,279.50000,710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5322.299969,541.468688
50%,558.00000,2320.000000,4.000000,2006.000000,1.000000,1.000000,2009.000000,6589.948470,678.667519
75%,836.50000,6875.000000,9.000000,2011.000000,1.000000,22.000000,2012.000000,7964.200644,866.203304
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000,21757.483418,3403.458599


### 添加根据 weekofday 计算的周X平均销售量

In [143]:
avgperday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6723 entries, 0 to 6722
Data columns (total 3 columns):
Store        6723 non-null int64
DayOfWeek    6723 non-null int64
AvgPerDay    6723 non-null float64
dtypes: float64(1), int64(2)
memory usage: 157.6 KB


In [144]:
# 合并avgperday
test_features = pd.merge(test,avgperday,on =['Store','DayOfWeek'],how ='left')
test_features.describe()

,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday,AvgPerDay
count,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,35292.000000
mean,20544.500000,555.899533,3.979167,0.854361,0.395833,0.443487,6841.206261
std,11861.228267,320.274496,2.015481,0.352748,0.489035,0.496802,2431.565183
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1351.291045
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000,5233.148148
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000,6491.439394
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000,7969.338710
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000,28450.328358


In [145]:
# 这里出错了
test_features.head()

#pd.concat([test_features, df4], axis=1, join_axes=[df1.index])

test_features = pd.merge(test_features,store,on = 'Store',how ='left')
test_features.describe()

,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday,AvgPerDay,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,store.MEAN(train.Sales),store.MEAN(train.Customers)
count,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,35292.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854361,0.395833,0.443487,6841.206261,5076.693925,4.429907,1264.787383,0.580607,14.182243,1168.078271,6834.578180,734.854654
std,11861.228267,320.274496,2.015481,0.352748,0.489035,0.496802,2431.565183,7221.221850,4.214484,969.984740,0.493466,16.177932,992.765386,2254.090836,352.537211
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1351.291045,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2790.380282,240.183099
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000,5233.148148,710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5307.296875,525.565968
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000,6491.439394,2410.000000,4.000000,2005.000000,1.000000,9.000000,2010.000000,6485.144278,658.733141
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000,7969.338710,6435.000000,9.000000,2011.000000,1.000000,31.000000,2012.000000,7782.973736,835.649872
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000,28450.328358,75860.000000,12.000000,2015.000000,1.000000,49.000000,2015.000000,20718.515924,3403.458599


In [146]:
list(test_features)

['Id',
 'Store',
 'DayOfWeek',
 'Date',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'AvgPerDay',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval',
 'store.MEAN(train.Sales)',
 'store.MEAN(train.Customers)']

In [147]:
test_features.drop({'Id','Date'},
              inplace = True, axis =1)


In [148]:
test_features = pd.get_dummies(test_features,
                         prefix=['StateHoliday', 'store.StoreType', 'store.Assortment','store.PromoInterval'], 
                         drop_first=True)

In [149]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 23 columns):
Store                                   41088 non-null int64
DayOfWeek                               41088 non-null int64
Open                                    41088 non-null float64
Promo                                   41088 non-null int64
SchoolHoliday                           41088 non-null int64
AvgPerDay                               35292 non-null float64
CompetitionDistance                     41088 non-null float64
CompetitionOpenSinceMonth               41088 non-null float64
CompetitionOpenSinceYear                41088 non-null float64
Promo2                                  41088 non-null int64
Promo2SinceWeek                         41088 non-null float64
Promo2SinceYear                         41088 non-null float64
store.MEAN(train.Sales)                 41088 non-null float64
store.MEAN(train.Customers)             41088 non-null float64
StateHoliday_a     

In [150]:
# Get missing columns in the test data
missing_cols = {'StateHoliday_b','StateHoliday_c'}

In [151]:
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    test_features[c] = 0
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 25 columns):
Store                                   41088 non-null int64
DayOfWeek                               41088 non-null int64
Open                                    41088 non-null float64
Promo                                   41088 non-null int64
SchoolHoliday                           41088 non-null int64
AvgPerDay                               35292 non-null float64
CompetitionDistance                     41088 non-null float64
CompetitionOpenSinceMonth               41088 non-null float64
CompetitionOpenSinceYear                41088 non-null float64
Promo2                                  41088 non-null int64
Promo2SinceWeek                         41088 non-null float64
Promo2SinceYear                         41088 non-null float64
store.MEAN(train.Sales)                 41088 non-null float64
store.MEAN(train.Customers)             41088 non-null float64
StateHoliday_a     

In [152]:
test_features =test_features.rename(columns={'CompetitionDistance':'store.CompetitionDistance',
                                              'CompetitionOpenSinceMonth':'store.CompetitionOpenSinceMonth',
                                              'CompetitionOpenSinceYear':'store.CompetitionOpenSinceYear',
                                              'Promo2':'store.Promo2',
                                              'Promo2SinceWeek':'store.Promo2SinceWeek',
                                              'Promo2SinceYear':'store.Promo2SinceYear'})
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 25 columns):
Store                                   41088 non-null int64
DayOfWeek                               41088 non-null int64
Open                                    41088 non-null float64
Promo                                   41088 non-null int64
SchoolHoliday                           41088 non-null int64
AvgPerDay                               35292 non-null float64
store.CompetitionDistance               41088 non-null float64
store.CompetitionOpenSinceMonth         41088 non-null float64
store.CompetitionOpenSinceYear          41088 non-null float64
store.Promo2                            41088 non-null int64
store.Promo2SinceWeek                   41088 non-null float64
store.Promo2SinceYear                   41088 non-null float64
store.MEAN(train.Sales)                 41088 non-null float64
store.MEAN(train.Customers)             41088 non-null float64
StateHoliday_a     

In [153]:
# Ensure the order of column in the test set is in the same order than in train set
test_features = test_features[features.columns]



In [154]:
# check if the test feature number is 25 or not
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 25 columns):
Store                                   41088 non-null int64
DayOfWeek                               41088 non-null int64
Open                                    41088 non-null float64
Promo                                   41088 non-null int64
SchoolHoliday                           41088 non-null int64
store.CompetitionDistance               41088 non-null float64
store.CompetitionOpenSinceMonth         41088 non-null float64
store.CompetitionOpenSinceYear          41088 non-null float64
store.Promo2SinceWeek                   41088 non-null float64
store.Promo2SinceYear                   41088 non-null float64
store.Promo2                            41088 non-null int64
store.MEAN(train.Sales)                 41088 non-null float64
store.MEAN(train.Customers)             41088 non-null float64
AvgPerDay                               35292 non-null float64
StateHoliday_a     

In [155]:
test_features.isnull().any()

Store                                   False
DayOfWeek                               False
Open                                    False
Promo                                   False
SchoolHoliday                           False
store.CompetitionDistance               False
store.CompetitionOpenSinceMonth         False
store.CompetitionOpenSinceYear          False
store.Promo2SinceWeek                   False
store.Promo2SinceYear                   False
store.Promo2                            False
store.MEAN(train.Sales)                 False
store.MEAN(train.Customers)             False
AvgPerDay                                True
StateHoliday_a                          False
StateHoliday_b                          False
StateHoliday_c                          False
store.StoreType_b                       False
store.StoreType_c                       False
store.StoreType_d                       False
store.Assortment_b                      False
store.Assortment_c                

In [156]:
len(test_features['AvgPerDay'])

41088

In [161]:
test_features.head()

,Store,DayOfWeek,Open,Promo,SchoolHoliday,store.CompetitionDistance,store.CompetitionOpenSinceMonth,store.CompetitionOpenSinceYear,store.Promo2SinceWeek,store.Promo2SinceYear,...,StateHoliday_b,StateHoliday_c,store.StoreType_b,store.StoreType_c,store.StoreType_d,store.Assortment_b,store.Assortment_c,"store.PromoInterval_Feb,May,Aug,Nov","store.PromoInterval_Jan,Apr,Jul,Oct","store.PromoInterval_Mar,Jun,Sept,Dec"
0,1,4,1.0,1,0,1270.0,9.0,2008.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,3,4,1.0,1,0,14130.0,12.0,2006.0,14.0,2011.0,...,0,0,0,0,0,0,0,0,1,0
2,7,4,1.0,1,0,24000.0,4.0,2013.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,8,4,1.0,1,0,7520.0,10.0,2014.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,9,4,1.0,1,0,2030.0,8.0,2000.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


### 注意到 AverPerDay 还处在空缺值

In [167]:
test_features['AvgPerDay']


test_features.isnull().sum(axis = 0)

Store                                      0
DayOfWeek                                  0
Open                                       0
Promo                                      0
SchoolHoliday                              0
store.CompetitionDistance                  0
store.CompetitionOpenSinceMonth            0
store.CompetitionOpenSinceYear             0
store.Promo2SinceWeek                      0
store.Promo2SinceYear                      0
store.Promo2                               0
store.MEAN(train.Sales)                    0
store.MEAN(train.Customers)                0
AvgPerDay                               5796
StateHoliday_a                             0
StateHoliday_b                             0
StateHoliday_c                             0
store.StoreType_b                          0
store.StoreType_c                          0
store.StoreType_d                          0
store.Assortment_b                         0
store.Assortment_c                         0
store.Prom

In [171]:
#df[pd.isnull(df).any(axis=1)]
test_features[pd.isnull(test_features).any(axis=1)]


,Store,DayOfWeek,Open,Promo,SchoolHoliday,store.CompetitionDistance,store.CompetitionOpenSinceMonth,store.CompetitionOpenSinceYear,store.Promo2SinceWeek,store.Promo2SinceYear,...,StateHoliday_b,StateHoliday_c,store.StoreType_b,store.StoreType_c,store.StoreType_d,store.Assortment_b,store.Assortment_c,"store.PromoInterval_Feb,May,Aug,Nov","store.PromoInterval_Jan,Apr,Jul,Oct","store.PromoInterval_Mar,Jun,Sept,Dec"
3424,1,7,0.0,0,0,1270.0,9.0,2008.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3425,3,7,0.0,0,0,14130.0,12.0,2006.0,14.0,2011.0,...,0,0,0,0,0,0,0,0,1,0
3426,7,7,0.0,0,0,24000.0,4.0,2013.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3427,8,7,0.0,0,0,7520.0,10.0,2014.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3428,9,7,0.0,0,0,2030.0,8.0,2000.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3429,10,7,0.0,0,0,3160.0,9.0,2009.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3430,11,7,0.0,0,0,960.0,11.0,2011.0,1.0,2012.0,...,0,0,0,0,0,0,1,0,1,0
3431,12,7,0.0,0,0,1070.0,0.0,0.0,13.0,2010.0,...,0,0,0,0,0,0,1,0,1,0
3432,13,7,0.0,0,0,310.0,0.0,0.0,45.0,2009.0,...,0,0,0,0,1,0,0,1,0,0
3433,14,7,0.0,0,0,1300.0,3.0,2014.0,40.0,2011.0,...,0,0,0,0,0,0,0,0,1,0


## 很奇怪全部都是 星期天的数据处在缺失

In [172]:
# 用0填充，这里值得再考虑
test_features.fillna(0,inplace = True)

## Testing on the final gradient descent boost model

In [173]:
# make predictions on the testing data
test_sales = model.predict(test_features)

In [174]:
test_sales

array([ 4999.45506508,  7860.64898541,  9532.68434885, ...,
        5873.26858049, 21898.44416938,  6519.45721577])

In [175]:
# output test results to csv file

df = pd.DataFrame({"Id":range(1,len(test_sales) + 1),'Sales':test_sales})
df.to_csv('submission_gboost_ft.csv',index = False)

## The final score on public leaderboard is ~0.14, which is not good enough

1. In the TPOT regeresor model, the light configuration is chosen since jupyter kernel will die on my local MBP

2. The light TPOT is chosen following instruction from https://github.com/EpistasisLab/tpot/issues/745 and https://github.com/EpistasisLab/tpot/issues/546

3. Therefore only simple model is fitted in light TPOT according to https://epistasislab.github.io/tpot/using/#built-in-tpot-configurations

4. Besides only 3 hours training time is given. All of those above factors may lead TPOT not find the best model

5. It is interesting to see what will happen on GCP with GPU and expanding training time and select normal TPOT.